# Классы в Python

## Классы и их экземпляры

Важно помнить, что в отличие от переменных, классы в Python принято называть через camlCase, т.е. с большой буквы. Простейший класс выглядит так:

In [1]:
class Human:
    pass

Вместо ключевого слова `pass` можно использовать `docstring`:

In [2]:
class Robot:
    """Данный класс позволяет создавать роботов"""

Чтобы посмотреть атрибуты и методы класса можно использовать функцию `dir()`:

In [3]:
print(dir(Robot))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


Надо отметить, что экземпляры классов хешируются, т.е. могут быть ключами в словаре.

Создать экземпляр класса довольно просто. Попробуем создать солнечную систему:

In [4]:
class Planet:
    
    count = 0
    
    def __new__(cls, *args, **kwargs):
        obj = super().__new__(cls)
        return obj
    
    def __init__(self, name, population=None):
        Planet.count += 1
        self.name = name
        self.population = population or []
        
    def __repr__(self):
        return f'Planet(\'{self.name}\')'
    
names = ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupyter', 'Saturn', 'Neptune', 'Pluto']
solar_system = [Planet(name) for name in names]

print(solar_system)

[Planet('Mercury'), Planet('Venus'), Planet('Earth'), Planet('Mars'), Planet('Jupyter'), Planet('Saturn'), Planet('Neptune'), Planet('Pluto')]


Как можно догадаться из кода выше, метод `__init__` инициализирует класс, а метод `__repr__` служит для того, чтобы отобразить в виде строки операцию создания объекта. Есть и менее очевидный момент: нам ничего не мешает удалить какой-то атрибут объекта, просто применив к нему оператор `del`:

```python
del solar_system[0].name
```

Возвращаясь к методу `__init__` надо отметить, что это не конструктор экземпляра класса, а именно инициализатор объекта. Он на входе уже получает готовый объект `self`. Конструктор экземпляра - это метод `__new__(cls, *args, **kwargs)`, и, если мы переопределяем его, то должны вернуть объект. Также, из кода выше следует, что наш класс является наследником класса `object`. Формально при вызове кода 

```python
planet = Planet('Earth')
```

происходит следующее:
```python
planet = Planet.__new__(Planet, 'Earth')

if isinstance(planet, Planet):
    Planet.__init__(planet, 'Earth')
```

Также в коде выше мы использовали атрибут класса `Planet` под названием `count`. Если привести аналогию с C#, то это по сути статический атрибут. Извне обратиться к такому атрибуту можно двумя способами: как через класс, так и через его объект:

In [5]:
print(Planet.count)
print(solar_system[0].count)

8
8


Надо понимать, что, как только число ссылок на объект класса достигает нуля, то срабатывает сборщик мусора и вызывается метод `__del__`, представляющий собой деструктор экземпляра класса. Ничто не мешает нам переопределить этот метод:

In [6]:
class Human:
    
    def __del__(self):
        print('Good bye!')
        
some_person = Human()
del some_person

Good bye!


Надо понимать, что переопределение метода `__del__` - это не самая лучшая практика, т.к. Python не гарантирует, что он всегда будет вызываться при завершении работы программы. Если мы хотим произвести какие-то действия при завершении работы с объектом, то лучше переопределить для этого метод `__exit__`. Но в данном случае придется еще переопределить и метод `__enter__`. Тогда мы сможем работать с объектом через контекстный менеджер.

Все пользовательские атрибуты класса можно посмотреть через специальный словарь `__dict__`:

In [7]:
solar_system[2].__dict__

{'name': 'Earth', 'population': []}

При этом надо понимать, что Python считает приватными все атрибуты, начинающиеся с двойного подчеркивания, т.е. такие атрибуты недоступны из классов-потомков. 

Название класса объекта можно получить через его атрибут `__class__`:

In [8]:
solar_system[2].__class__

__main__.Planet

## Методы

Понятно, что методы - это функции, действующие в контексте экземпляра класса и способные менять его состояние.

In [9]:
class Human:
    def __init__(self, name, age = 0):
        self.name = name
        self.age = age
        
class Planet:
    def __init__(self, name, population=None):
        self.name = name
        self.population = population or []
        
    def add_human(self, human):
        print(f'Welcome to {self.name}, {human.name}!')
        self.population += [human]
        
mars = Planet('Mars')
bob = Human('Bob')
mars.add_human(bob)

Welcome to Mars, Bob!


Теперь посмотрим, что такое методы класса.

In [10]:
class Event:
    def __init__(self, description, event_date):
        self.description = description
        self.event_date = event_date
    
    def __repr__(self):
        return f'Event(\'{self.description}\', date.fromisoformat(\'{self.event_date}\'))'
        
    def __str__(self):
        return f'Event \'{self.description}\' as {self.event_date}.'
    
    @classmethod
    def create(cls, description, event_date):
        return cls(description, event_date)
    
from datetime import date
Event.create('я начал работать в Teradata', date.fromisoformat('2021-04-05'))

Event('я начал работать в Teradata', date.fromisoformat('2021-04-05'))

Как видно выше, в метод класса передается параметр `cls`, а не `self` и такой метод имеет декоратор `@classmethod`. 

А теперь посмотрим на статические методы:

In [11]:
class Human:
    
    def __init__(self, name, age=0):
        self.name = name
        self.age = age
        
    @staticmethod
    def is_age_valid(age):
        return 0 <= age <= 150

К статическим методам можно обращаться из экземпляра класса.

А еще в Python есть свойства:

In [12]:
class Robot:
    def __init__(self, power):
        self.power = power
        
    power = property()
    
    @power.setter
    def power(self, value):
        self._power = value if value >= 0 else 0
        
    @power.getter
    def power(self):
        return self._power
    
    @power.deleter
    def power(self):
        print('The robot is useless now!')
        del self._power
        
r = Robot(-100)
r.power = -10
print(r.power)

0


Выше показан комплексный случай, когда мы обрамляем и геттер и сеттер. Если же нам надо лишь фиксировать считывание атрибута, то достаточно использовать декоратор `@property` перед методом объекта, который и будет делать какую-то полезную работу.

## Наследование

Начнем с простого примера наследования:

In [13]:
class Pet:
    def __init__(self, name=None):
        self.name = name
        
class Dog(Pet):
    def __init__(self, name, breed=None):
        super().__init__(name)
        self.breed = breed
        
    def say(self):
        return f'{name}: гав!'

А теперь разберем пример посложнее с реализацией множественного наследования:

In [14]:
import json

class ExportJSON:
    def to_json(self):
        return json.dumps(self.__dict__)
    
class ExDog(Dog, ExportJSON):
    pass

dog = ExDog(name='Белка', breed='Дворняжка')
print(dog.to_json())

{"name": "\u0411\u0435\u043b\u043a\u0430", "breed": "\u0414\u0432\u043e\u0440\u043d\u044f\u0436\u043a\u0430"}


Как уже говорилось, любой класс Python является потомком класса `object` (не обязательно прямым). Посмотрим иерархию классов на примере `ExDog`:

In [15]:
def get_hierarchy(cls, intendation=0):
    result = '----' * intendation + cls.__name__
    if cls.__name__ != 'object':  
        intendation += 1
        for basic_cls in cls.__bases__:
            result += ('\n' + get_hierarchy(basic_cls, intendation))
    return result

print('\n'.join(list(reversed(get_hierarchy(ExDog).split('\n')))))
#print(get_hierarchy(ExDog))

--------object
----ExportJSON
------------object
--------Pet
----Dog
ExDog


In [16]:
def get_class_hierarchy(base_cls: type, intendation: int = 0, clss = []) -> str:
    if len(clss) > 0 and base_cls.__name__ not in clss:
        return ''
    result = '\n' * (intendation > 0) + '-' * intendation + base_cls.__name__
    if len(base_cls.__subclasses__()) != 0:
        for subclass in base_cls.__subclasses__():
            result += get_class_hierarchy(subclass, intendation + 3, clss)
    return result

print(get_class_hierarchy(base_cls=object, clss=['object', 'ExportJSON', 'Dog', 'Pet', 'ExDog']))

object
---Pet
------Dog
---------ExDog
---ExportJSON
------ExDog


Как видно из кода выше, базовые классы можно получить через атрибут `__bases__` текущего класса. Получить класс любого объекта можно функцией `type()`. Проверить принадлежность объекта к классу можно с помощью функции `isinstance()`, а наличие одного класса в числе родителей другого класса можно функцией `issubclass()`.

Также, выше видно, что у родителей класса `ExDog` довольно замысловатая структура, поэтому не очень очевидно, метод какого класса будет выполнен и вызове метода, определенного в нескольких классах. Для формализации поиска таких методов существует такая вещь, как method resulution order (mro). В классе `ExDog` он следующий:

In [17]:
ExDog.__mro__

(__main__.ExDog, __main__.Dog, __main__.Pet, __main__.ExportJSON, object)

Важно понимать, что чем ближе класс расположен к началу списка родителей, тем скорее будут искаться методы в нем и его родителях. Также весьма очевидным является тот факт, что вызывая метод `super()` в методе `__init__()` класса-потомка, мы обращаемся к непосредственному родителю. Менее очевиден момент, что похожим образом мы можем обратиться к "дедушке", указав в методе `super()` класс непосредственного потомка в качестве параметра:

In [18]:
class ExDog(Dog, ExportJSON):
    def __init__(self, name, breed=None):
        super().__init__(name, breed)
    
class WoolenDog(Dog, ExportJSON):
    def __init__(self, name, breed=None):
        super(Dog, self).__init__(name)
        self.breed = f'Шерстяная собака породы {breed}'
dog = WoolenDog('Жучка', breed='Такса')
print(dog.breed)

Шерстяная собака породы Такса


## Композиция классов

Идея композиции очень проста и очевидна: если мы хотим лишь пользоваться каким-то классом, никак не переопределяя его поведения, то надо включить объект этого класса в создаваемый нами класс, а не наследовать его.

In [19]:
import json
import dicttoxml

class PetExport:
    def export(self, dog):
        raise NotImplementedError
        
class ExportJSON(PetExport):
    def export(self, dog):
        return json.dumps({key: val for key, val in dog.__dict__.items() if not key.startswith('__')})
    
class ExportXML(PetExport):
    def export(self, dog):
        return dicttoxml.dicttoxml({key: val for key, val in dog.__dict__.items() if not key.startswith('__')})
    
class Pet:
    def __init__(self, name):
        self.name = name
        
class Dog(Pet):
    def __init__(self, name, breed=None):
        super().__init__(name)
        self.breed = breed
        
class ExDog(Dog):
    def __init__(self, name, breed=None, exporter=None):
        super().__init__(name, breed)
        self.__exporter__ = exporter or ExportJSON()
        
    def export(self):
        return self.__exporter__.export(self)
    
dog = ExDog(name='Веста', breed='Французский бульдог', exporter=ExportXML())
dog.export()

b'<?xml version="1.0" encoding="UTF-8" ?><root><name type="str">\xd0\x92\xd0\xb5\xd1\x81\xd1\x82\xd0\xb0</name><breed type="str">\xd0\xa4\xd1\x80\xd0\xb0\xd0\xbd\xd1\x86\xd1\x83\xd0\xb7\xd1\x81\xd0\xba\xd0\xb8\xd0\xb9 \xd0\xb1\xd1\x83\xd0\xbb\xd1\x8c\xd0\xb4\xd0\xbe\xd0\xb3</breed></root>'